In [2]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

# ==========================================
# STEP 1: LOAD AND ENCODE STUDENT IMAGES
# ==========================================
path = 'students'
images = []
classNames = []
myList = os.listdir(path)

print(f'Found {len(myList)} files in {path} folder.')

for cl in myList:
    # Filter for image files only
    if cl.lower().endswith(('.png', '.jpg', '.jpeg')):
        curImg = cv2.imread(f'{path}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # This uses the CNN-based model to generate 128 face embeddings
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

print('Encoding started... please wait.')
encodeListKnown = findEncodings(images)
print('Encoding Complete. Total faces encoded:', len(encodeListKnown))

# ==========================================
# STEP 2: ATTENDANCE LOGGING FUNCTION
# ==========================================
def markAttendance(name):
    # Ensure file exists or create it
    if not os.path.exists('Attendance.csv'):
        with open('Attendance.csv', 'w') as f:
            f.writelines('Name,Time,Date')
            
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        
        if name not in nameList:
            now = datetime.now()
            timeString = now.strftime('%H:%M:%S')
            dateString = now.strftime('%d-%m-%Y')
            f.writelines(f'\n{name},{timeString},{dateString}')
            print(f"Attendance marked for: {name}")

# ==========================================
# STEP 3: REAL-TIME WEBCAM RECOGNITION
# ==========================================
# 0 is usually the built-in FaceTime HD camera on Mac
cap = cv2.VideoCapture(0)

print("Starting Webcam... Point the camera at your face.")
print("Press 'q' to exit.")

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    # 1. Resize for performance (M1 can handle larger, but 0.25 is very smooth)
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # 2. Detect face locations and encodings
    # 'hog' is fast, 'cnn' is highly accurate but slower
    facesCurFrame = face_recognition.face_locations(imgS, model='hog')
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    # 3. Compare detected faces with known encodings
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=0.5)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        if len(faceDis) > 0:
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                
                # 4. Draw bounding box and label
                # Since we resized to 1/4th, we must multiply coordinates by 4
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
                
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2-35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
                
                # 5. Log the entry
                markAttendance(name)

    # Display the result
    cv2.imshow('M1 Attendance Monitor', img)
    
    # Check for 'q' key to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ==========================================
# STEP 4: CLEANUP (Critical for MacOS)
# ==========================================
cap.release()
cv2.destroyAllWindows()
# Mac-specific fix for frozen windows
for i in range(5):
    cv2.waitKey(1)

/Users/arif/miniforge3/envs/tf/lib/python3.10/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Found 4 files in students folder.
Encoding started... please wait.
Encoding Complete. Total faces encoded: 3
Starting Webcam... Point the camera at your face.
Press 'q' to exit.
Attendance marked for: TAHMID


In [3]:
import os

path = 'students'

# Create the folder if it doesn't exist
if not os.path.exists(path):
    os.makedirs(path)
    print(f"Directory '{path}' created successfully!")
else:
    print(f"Directory '{path}' already exists.")

Directory 'students' already exists.


In [4]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

# --- 1. SETUP LOGGING ---
log_file = "attendance_log.txt"
# This dictionary tracks when a student was last logged to prevent duplicates
last_logged = {} 
COOLDOWN_SECONDS = 30 

def mark_attendance_txt(name):
    now = datetime.now()
    current_time = now.strftime('%Y-%m-%d %H:%M:%S')
    
    # Check if we should log this person again
    if name not in last_logged or (now - last_logged[name]).total_seconds() > COOLDOWN_SECONDS:
        with open(log_file, "a") as f:
            f.write(f"NAME: {name:15} | TIMESTAMP: {current_time}\n")
        
        last_logged[name] = now
        print(f"Logged to TXT: {name} at {current_time}")

# --- 2. LOAD AND ENCODE ---
path = 'students'
if not os.path.exists(path):
    os.makedirs(path)

images = []
classNames = []
myList = [f for f in os.listdir(path) if not f.startswith('.')]

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    if curImg is not None:
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodes = face_recognition.face_encodings(img)
        if encodes:
            encodeList.append(encodes[0])
    return encodeList

print("Encoding known faces... please wait.")
encodeListKnown = findEncodings(images)
print(f'Encoding Complete. Loaded {len(encodeListKnown)} faces.')

# --- 3. WEBCAM LOOP ---
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success: break

    # Pre-processing for the CNN
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Face Detection and Encoding
    facesCurFrame = face_recognition.face_locations(imgS, model='hog')
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=0.5)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        if len(faceDis) > 0:
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                
                # Draw Visuals
                y1, x2, y2, x1 = [v * 4 for v in faceLoc]
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                # Log to TXT
                mark_attendance_txt(name)

    cv2.imshow('Attendance System (Press Q to Quit)', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
# Mac cleanup
for i in range(5): cv2.waitKey(1)

Encoding known faces... please wait.
Encoding Complete. Loaded 3 faces.
Logged to TXT: ARIFUL_ISLAM at 2025-12-21 13:13:12
